# About this code
This is the start of an enumerator for the valid Congressional districting plans of the state of Iowa.  Iowa (currently) is apportioned four Congressional districts and the state laws prohibit *the splitting of counties* in forming such plans.  Since there are 99 counties in Iowa arranged in a roughly grid-like pattern and we have had [some success in enumerating partitions of grid graphs](https://github.com/zschutzman/enumerator), it seems a natural question to ask whether or not we can do the same for Iowa.  Curious readers are encouraged to take a look at the enumeration strategy for the grid graph, as this setting, which is only a slight modification, provides an excellent test case for the limitations of that method.

New in this setting is the population constraint.  Legally, each district must have (nearly) the same number of people.  We have population data for the counties of Iowa and we need to endow the graph with that information in order to evaluate whether a districting plan is legal or not.

At a high level, the strategy for enumeration will be the same.  We'll first enumerate all of the valid individual districts, where validity is determined by having the appropriate population, being geographically connected, and not creating any unfillable holes.

## A Warning:

This code is essentially optimal up to implementation details, but it still requires an enormous amount of time to run.  This notebook is illustrative of the algorithm  and a proof of its correctness rather than something you should be running yourself.


The expository details in this notebook are lighter than in the previous one. Here we seek to outline the proof-of-correctness of the algorithm rather than a tutorial on the enumeration of partitions.

Users who *really* want to run this code should download it and run it from a Julia terminal.  This can be done  

In [ ]:
# These are the packages we need to use.  If you don't
# have one, uncomment and run in the next block the
# appropriate lines and then run this block again
using Pkg
using LightGraphs, MetaGraphs
using IterTools
using DataStructures
using Base
using Combinatorics
using StatsBase

In [ ]:
Pkg.add("LightGraphs")
Pkg.add("IterTools")
Pkg.add("DataStructures")
Pkg.add("Combinatorics")
Pkg.add("MetaGraphs")
Pkg.add("StatsBase")

In [ ]:
# make_iowa_graph() is defined in iowa_graph.jl
# it is built from a MetaGraph construction and 
# has population data at each node

include("iowa_graph.jl")
const iagr = make_iowa_graph()

In [ ]:
# since dictionary queries are faster than
# MetaGraph queries, we'll drop the adjacency
# information into a dictionary and declare it 
# a constant

ia_d = DefaultDict(Set)

for e in edges(iagr)
	push!(ia_d[src(e)], dst(e))
	push!(ia_d[dst(e)], src(e))
end

const ia_dict = ia_d


    

In [ ]:


# throughout, a (sub)graph will be represented
# by a bit vector. Each node in the graph
# is named with a number 1-99, so the ith 
# bit in the vector indicates whether or 
# not node i is in the subgraph




# this function tells you the population of a 
# collection of nodes in the graph.
function distpop(om)
	g = induced_subgraph(iagr, [i for i in findall(x->x == 1,om)])[1]
	return sum( get_prop(g,v,:pop) for v in vertices(g)   )
end


# this function checks to see if a subgraph
# creates an unfillable hole in the graph
# if so, this subgraph cannot be a valid district,
# since there can't be other districts to fill 
# the gaps
function iowa_holes(om,pop_tol)
	g = induced_subgraph(iagr, [i for i in findall(x->x == 0,om)])[1]
	for c in connected_components(g)
		if sum( get_prop(g,v,:pop) for v in c  )  < pop_tol[1]
			return false  
		end
	end
	return true
end





# this does a recursive depth-first search of the 
# set of all subgraphs to pick out the ones which 
# are valid districts
function recurse_dfs(om, forbid, pop_tol, outf,minm,maxm)

    
    # base case: if the population of the subgraph
    # is too large, terminate the branch, since
    # adding more counties can't bring down the population
	if distpop(om) > pop_tol[2]
		return

        
    # if the population is in the tolerated range, we found
    # a valid district, so write it to the file
	elseif pop_tol[1] <= distpop(om) <= pop_tol[2] && iowa_holes(om,pop_tol)
		str = replace(string(om), "," => "")
		str = replace(str, "Int8" => "")
		str = replace(str, "[" => "")
		str = replace(str, "]" => "")
		str = replace(str, " " => "")
		write(outf,"$(str)\n")
		global found
		found +=1
		print("found:  ",found,"                  ",'\r')
	end


    # given a subgraph, we can list the 
    # nodes which can be legally added 
    # to make a slightly larger subgraph
    
    # these nodes should be:
        # adjacent to something in the graph
        # have a higher id than minm, the starting position
        # not be in the set of forbidden vertices
	cands = Set()
	for j=minm:maxm
		if om[j] == 1
			for n in ia_dict[j]
				if n>minm && om[n] ==0 && n ∉ forbid
					push!(cands,n)
				end
			end
		end

	end

    
    # a vertex will be forbidden if
    # we have already (recursively) tried
    # to explore its branch.  this stops
    # us from doing extra work, since we 
    # don't want to explore adding vertex #5
    # and then #14 on one branch and the 
    # opposite on some other branch
    
    # if we have a candidate that is illegal to 
    # add since it creates a hole, we "defer" on 
    # it -- we don't explore its branch, but we 
    # also don't forbid it from being used later
    newforbid = Set(forbid)
	for c in cands
		om[c] = 1
		if iowa_holes(om,pop_tol)
			recurse_dfs(om,newforbid,pop_tol,outf,minm,max(c,maxm))
			push!(newforbid,c)

			
		end
		om[c] = 0

	end


end





function enumerate2()
	global found

	for i=67:-1:67
		found = 0

		print('\n',i,'\n')
		outf = open("ia_dists_pm500/dfs2-s2$i.txt","w")
		s = vec(zeros(Int8,1,99))
		s[i] = 1
		recurse_dfs(s,Set(),[761000,762000],outf,i,i)
		close(outf)
	end
end



@time enumerate2()